# 3.1 Tickers
The goal is to solve the problems of the Polygon ticker lists in the introduction. Before we do that we will download the ticker list for all days from Polygon and store them into the map <code>tickers</code>.

In [20]:
###
from polygon.rest import RESTClient
from datetime import datetime, date, time, timedelta
from pytz import timezone
from functools import lru_cache
from utils import get_market_dates, first_trading_date_after_equal, last_trading_date_before_equal, get_tickers
import os
import pytz
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import mplfinance as mpf

POLYGON_DATA_PATH = "../data/polygon/"

START_DATE = date(2019, 1, 1)
END_DATE = date(2023, 9, 7)

with open(POLYGON_DATA_PATH + "secret.txt") as f:
    KEY = next(f).strip()

client = RESTClient(api_key=KEY)

First, I will create a function to download the ticker for a specific date.

In [2]:
###
def download_tickers(date_):
    """Retrieve the ticker list for a specific date

    Args:
        date_ (Date): the Date for which to download the ticker list

    Returns:
        DataFrame: the ticker list
    """
    
    date_iso = date_.isoformat()

    ticker_list_iterator_active = client.list_tickers(type="CS", date=date_iso, active=True, market='stocks', limit=1000)
    ticker_list_iterator_delisted = client.list_tickers(type="CS", date=date_iso, active=False, market='stocks', limit=1000)
    ticker_list_iterator_active_adr = client.list_tickers(type="ADRC", date=date_iso, active=True, market='stocks', limit=1000)
    ticker_list_iterator_delisted_adr = client.list_tickers(type="ADRC", date=date_iso, active=False, market='stocks', limit=1000)
    tickers_active = pd.DataFrame(ticker_list_iterator_active)
    tickers_delisted = pd.DataFrame(ticker_list_iterator_delisted)
    tickers_active_adr = pd.DataFrame(ticker_list_iterator_active_adr)
    tickers_delisted_adr = pd.DataFrame(ticker_list_iterator_delisted_adr)

    tickers_all = pd.concat([tickers_active, tickers_delisted, tickers_active_adr, tickers_delisted_adr])
    tickers_all.sort_values(by = "ticker", inplace=True)
    tickers_all.reset_index(inplace=True, drop=True)
    return tickers_all[['ticker', 'name', 'active', 'delisted_utc', 'last_updated_utc', 'cik', 'composite_figi', 'type']]

Then all ticker lists are downloaded and stored in the <code>raw/tickers/</code> map. But only the one that we need if we already have some.

In [3]:
###
# Get a list of what we already have
files = os.listdir(POLYGON_DATA_PATH + f'raw/tickers')
available_dates = [date.fromisoformat(file.replace(".csv", "")) for file in files]

trading_dates = get_market_dates()
for day in trading_dates:
    # Only download what we do not have
    if day >= START_DATE and day <= END_DATE and day not in available_dates:
        tickers = download_tickers(day)
        tickers.to_csv(POLYGON_DATA_PATH + f"raw/tickers/{day.isoformat()}.csv")
        print(f"Downloaded tickers for {day.isoformat()}")

Downloaded tickers for 2023-09-05
Downloaded tickers for 2023-09-06
Downloaded tickers for 2023-09-07


In [13]:
trading_dates[-1]

datetime.date(2023, 9, 1)

A random ticker list:

In [4]:
pd.read_csv(POLYGON_DATA_PATH + f"raw/tickers/2022-06-09.csv", index_col=0).head(3)

,ticker,name,active,delisted_utc,last_updated_utc,cik,composite_figi,type
0,A,Agilent Technologies Inc.,True,NaN,2022-06-14T00:00:00Z,1090872.0,BBG000C2V3D6,CS
1,AA,"Alcoa, Inc.",False,2016-11-01T00:00:00Z,2016-11-01T00:00:00Z,4281.0,NaN,CS
2,AA,ALCOA INC,False,2016-10-07T00:00:00Z,2016-10-07T00:00:00Z,4281.0,NaN,CS


We observe that the <code>last_updated_utc</code> does not match the date of the ticker list. For example for "A", this date is *after* 2022-06-09. So this value is not point-in-time. So this value is useless for us. Neither do we need <code>delisted_utc</code>, because we will determine the <code>end_date</code> by the ticker lists themselves. We will also determine the <code>start_date</code>, which Polygon does not give at all.

Later when we do have data, we will create a new column <code>start_data</code> and <code>end_data</code> which gives the start and end dates from the available data.

# 3.2 Building the tickers loop
Now we can finally create our ticker list, which includes all tickers. The process involves looping over all Polygon ticker lists and updating our own one. First some notation: T is our ticker list that we iteratively update using Polygons ticker list. P(i) is the Polygon ticker list from day *i*. 

1. On day 1, our ticker list is the same as the one from Polygon, but with some extra columns. We create a column <code>start_date</code> which is day 1 and <code>end_date</code> with is empty. We are only interested in stocks that were active on that day.
2. For all *i = 2 ... n* days, for the active stocks:
    * **Delistings**: The stocks that are in T but not in P(i) are the stocks that are removed by Polygon (e.g. FB). For these tickers we set the <code>end_date</code> in T to day *i-1*. 
    * **New listings**: The stocks that are in P(i) but not in T are the new listings. We will append the new stock to T and set the start_date to day *i*.
    * **Everything else**: The stocks that are both in P(i) and T are the stocks that 'continue their listings'. We do nothing.

Two tickers are the 'same' if all fields except <code>last_updated_utc</code> or <code>delisted_utc</code> are the same.

For testing, we will start with 2022-06-08 and update to 2022-06-09. Both FB and META should then be included with the correct start and end dates. The start and end date of FB should be 2022-06-08 and the start date of META should be 2022-06-09. The end date of META should be empty.

In [5]:
day_1 = date(2022, 6, 8)
day_2 = date(2022, 6, 9)

our_tickers = pd.read_csv(
    POLYGON_DATA_PATH + f"raw/tickers/{day_1.isoformat()}.csv",
    index_col=0,
)
our_tickers = our_tickers[["ticker", "name", "active", "cik", "composite_figi", "type"]]
our_tickers = our_tickers[our_tickers["active"] == True]
our_tickers.reset_index(inplace=True, drop=True)

our_tickers["start_date"] = day_1
our_tickers["end_date"] = pd.NaT

tickers_day_2 = pd.read_csv(
    POLYGON_DATA_PATH + f"raw/tickers/{day_2.isoformat()}.csv",
    index_col=0,
)
tickers_day_2 = tickers_day_2[["ticker", "name", "active", "cik", "composite_figi", "type"]]
tickers_day_2 = tickers_day_2[tickers_day_2["active"] == True]
tickers_day_2.reset_index(inplace=True, drop=True)

In [6]:
our_tickers.head(2)

,ticker,name,active,cik,composite_figi,type,start_date,end_date
0,A,Agilent Technologies Inc.,True,1090872.0,BBG000C2V3D6,CS,2022-06-08,NaT
1,AA,Alcoa Corporation,True,1675149.0,BBG00B3T3HD3,CS,2022-06-08,NaT


In [7]:
tickers_day_2.head(2)

,ticker,name,active,cik,composite_figi,type
0,A,Agilent Technologies Inc.,True,1090872.0,BBG000C2V3D6,CS
1,AA,Alcoa Corporation,True,1675149.0,BBG00B3T3HD3,CS


Preliminary check for duplicates

In [8]:
# Preliminary check: no duplicates
if our_tickers[["ticker", "name", "active", "cik", "composite_figi", "type"]].duplicated().all():
    raise Exception("There are duplicates!")

if tickers_day_2[["ticker", "name", "active", "cik", "composite_figi", "type"]].duplicated().all():
    raise Exception("There are duplicates!")

We will first get the delisting and new listings. (Nothing has to be done with the kept listings).

In [9]:
# DELISTINGS: Get tickers that are in T but not in P(2). This is actually not straightforward (https://stackoverflow.com/questions/28901683/pandas-get-rows-which-are-not-in-other-dataframe). We need to get the rows in tickers_day_2 that are not in our_tickers. We will use the merge function but specifying indicator=True and use a left merge (tickers_day_2 left merge to our_tickers). What gets returned is a dataframe with the flags "left_only", "right_only" and "both". If the indicator is "left_only", it means that it existed in only in the left DataFrame (our_tickers). This is exactly what we need. 
indicator_delisted = our_tickers[["ticker", "name", "active", "cik", "composite_figi", "type"]].merge(tickers_day_2[["ticker", "name", "active", "cik", "composite_figi", "type"]], on=["ticker", "name", "active", "cik", "composite_figi", "type"], 
                   how='left', indicator=True)
indicator_delisted = indicator_delisted["_merge"] # Only get the indicator

delisted_tickers = our_tickers[indicator_delisted == "left_only"] # Only get the delisted tickers

# NEW LISTINGS: Swap the DataFrames
indicator_new = tickers_day_2[["ticker", "name", "active", "cik", "composite_figi", "type"]].merge(our_tickers[["ticker", "name", "active", "cik", "composite_figi", "type"]], on=["ticker", "name", "active", "cik", "composite_figi", "type"], 
                   how='left', indicator=True)
indicator_new = indicator_new["_merge"]
new_tickers = tickers_day_2[indicator_new == "left_only"]

# KEPT LISTINGS
current_tickers = our_tickers[indicator_delisted == "both"]
# current_tickers = tickers_day_2[indicator_new == "both"] # It does not matter which one we choose

In [10]:
print(len(delisted_tickers))
delisted_tickers.head(2)

6


,ticker,name,active,cik,composite_figi,type,start_date,end_date
1144,CERN,Cerner Corp,True,804753.0,BBG000BFDLV8,CS,2022-06-08,NaT
1799,DYNS,Dynamics Special Purpose Corp. Class A Common ...,True,1854270.0,BBG010WX7ZB3,CS,2022-06-08,NaT


In [11]:
print(len(new_tickers))
new_tickers.head(2)

5


,ticker,name,active,cik,composite_figi,type
2435,GLAQ,Globis Acquisition Corp. common stock,True,1823383.0,NaN,CS
2725,HOUS,Anywhere Real Estate Inc.,True,1398987.0,BBG000QN4GY3,CS


In [12]:
len(current_tickers)

6275

Then we will process the delistings and listings.

In [13]:
# DELISTINGS: register delisting date and set to inactive.
our_tickers.loc[indicator_delisted == "left_only", "end_date"] = day_1 #Not day_2!
our_tickers.loc[indicator_delisted == "left_only", "active"] = False

our_tickers[our_tickers["ticker"] == "FB"]

,ticker,name,active,cik,composite_figi,type,start_date,end_date
2080,FB,"Meta Platforms, Inc. Class A Common Stock",False,1326801.0,BBG000MM2P62,CS,2022-06-08,2022-06-08


In [14]:
# NEW LISTINGS: append the new tickers and register start date
print(len(our_tickers))
print(len(new_tickers))

our_tickers = pd.concat([our_tickers, new_tickers])
our_tickers.reset_index(inplace=True, drop=True)
our_tickers['start_date'].fillna(value=day_2, inplace=True)

print(len(our_tickers))
our_tickers[our_tickers["ticker"] == "META"]

6281
5
6286


,ticker,name,active,cik,composite_figi,type,start_date,end_date
6283,META,"Meta Platforms, Inc. Class A Common Stock",True,1326801.0,BBG000MM2P62,CS,2022-06-09,NaN


Some final checks and setting <code>end_date</code> for the active listings at END_DATE.

In [15]:
if our_tickers[["ticker", "name", "active", "type", "start_date"]].isnull().values.any():
    raise Exception("There are missing values.")

# After all is done, set the end_date for active stocks to the new day. This is only done after all iterations. 
our_tickers["end_date"].fillna(value=day_2, inplace=True)

The result is correct. FB is included with the correct <code>end_date</code>. Then META starts with the correct <code>start_date</code>.

In [16]:
our_tickers[our_tickers['ticker'].isin(['FB', 'META'])]

,ticker,name,active,cik,composite_figi,type,start_date,end_date
2080,FB,"Meta Platforms, Inc. Class A Common Stock",False,1326801.0,BBG000MM2P62,CS,2022-06-08,2022-06-08
6283,META,"Meta Platforms, Inc. Class A Common Stock",True,1326801.0,BBG000MM2P62,CS,2022-06-09,2022-06-09


# 3.3 The tickers loop
Putting it all in a loop gives the following code. We save the results to <code>tickers_v1.csv</code>.

In [ ]:
market_days = get_market_dates()

first_trading_date_after_start_date = first_trading_date_after_equal(START_DATE)
last_trading_date_before_end_date = last_trading_date_before_equal(END_DATE)

for day in market_days:
    if day == first_trading_date_after_start_date:
        # At the start, our ticker list is the same as polygon.
        our_tickers = pd.read_csv(
            POLYGON_DATA_PATH + f"raw/tickers/{first_trading_date_after_start_date.isoformat()}.csv",
            index_col=0,
            keep_default_na=False,
            na_values=['#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND', '1.#QNAN', '<NA>', 'N/A', 'NULL', 'NaN', 'None', 'n/a', 'nan', 'null']
        ) #There is a stock named 'NA'. We have to avoid pandas treating it as a N/A value.
        our_tickers = our_tickers[["ticker", "name", "active", "cik", "composite_figi", "type"]]
        our_tickers = our_tickers[our_tickers["active"] == True]
        our_tickers.reset_index(inplace=True, drop=True)
    
        # Initialize tickers_all
        our_tickers["start_date"] = START_DATE
        our_tickers["end_date"] = pd.NaT

    elif day > START_DATE and day <= END_DATE:
        # Get new ticker list to update ours
        tickers_day_i = pd.read_csv(
            POLYGON_DATA_PATH + f"raw/tickers/{day.isoformat()}.csv",
            index_col=0,
            keep_default_na=False,
            na_values=['#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND', '1.#QNAN', '<NA>', 'N/A', 'NULL', 'NaN', 'None', 'n/a', 'nan', 'null']
        )
        tickers_day_i = tickers_day_i[["ticker", "name", "active", "cik", "composite_figi", "type"]]
        tickers_day_i = tickers_day_i[tickers_day_i["active"] == True]
        tickers_day_i.reset_index(inplace=True, drop=True)

        # Preliminary check: no duplicates
        if our_tickers[["ticker", "name", "active", "cik", "composite_figi", "type"]].duplicated().all():
            raise Exception("There are duplicates!")

        if tickers_day_i[["ticker", "name", "active", "cik", "composite_figi", "type"]].duplicated().all():
            raise Exception("There are duplicates!")

        # DELISTINGS
        indicator_delisted = our_tickers[["ticker", "name", "active", "cik", "composite_figi", "type"]].merge(tickers_day_i[["ticker", "name", "active", "cik", "composite_figi", "type"]], on=["ticker", "name", "active", "cik", "composite_figi", "type"], how='left', indicator=True)

        indicator_delisted['_merge'] = np.where(our_tickers["active"], indicator_delisted['_merge'], "both") # ERROR FIX: If in our ticker list we have already set it inactive, it should not be added to the list of delisted stocks again. By setting _merge to "both" we skip the already inactive stocks.

        indicator_delisted = indicator_delisted["_merge"] # Only get the indicator
        delisted_tickers = our_tickers[indicator_delisted == "left_only"]

        # NEW LISTINGS
        indicator_new = tickers_day_i[["ticker", "name", "active", "cik", "composite_figi", "type"]].merge(our_tickers[["ticker", "name", "active", "cik", "composite_figi", "type"]], on=["ticker", "name", "active", "cik", "composite_figi", "type"], 
                        how='left', indicator=True)
        indicator_new = indicator_new["_merge"]
        new_tickers = tickers_day_i[indicator_new == "left_only"]

        # PROCESS DELISTINGS
        previous_day = market_days[market_days.index(day) - 1] # Getting previous trading day
        our_tickers.loc[indicator_delisted == "left_only", "end_date"] = previous_day
        our_tickers.loc[indicator_delisted == "left_only", "active"] = False
        
        # PROCESS NEW LISTINGS
        our_tickers = pd.concat([our_tickers, new_tickers])
        our_tickers.reset_index(inplace=True, drop=True)
        our_tickers['start_date'].fillna(value=day, inplace=True)

        # Final checks
        if our_tickers[["ticker", "name", "active", "type", "start_date"]].isnull().values.any():
            #null_data = our_tickers[our_tickers[["ticker", "name", "active", "type", "start_date"]].isnull().any(axis=1)]
            raise Exception("There are missing values.")
        
        print(f'{day.isoformat()}: Amount of stocks {len(our_tickers)}')
        
        # Finalize
        if day == last_trading_date_before_end_date:
            our_tickers["end_date"].fillna(value=last_trading_date_before_end_date, inplace=True)
            our_tickers = our_tickers.sort_values(by=["ticker", "end_date"]).reset_index(drop=True)
            our_tickers[["ticker", "name", "active", "start_date", "end_date", "type", "cik", "composite_figi"]].to_csv("../data/tickers_v1.csv")

We also create a function to retrieve the ticker list.

In [18]:
###
def get_tickers(v=5):
    """
    Retrieve the ticker list. Default is 5.
    """
    tickers = pd.read_csv(
        f"../data/tickers_v{v}.csv",
        parse_dates=["start_date", "end_date"],
        index_col=0,
        keep_default_na=False,
        na_values=["#N/A","#N/AN/A","#NA","-1.#IND","-1.#QNAN","-NaN","-nan","1.#IND","1.#QNAN","<NA>","N/A","NULL","NaN","None","n/a","nan","null",],
    )
    tickers["start_date"] = pd.to_datetime(tickers["start_date"]).dt.date
    tickers["end_date"] = pd.to_datetime(tickers["end_date"]).dt.date

    # This will only be applied in future notebooks.
    if tickers.columns.isin(["start_data", "end_data"]).any():
        tickers["start_data"] = pd.to_datetime(tickers["start_data"]).dt.date
        tickers["end_data"] = pd.to_datetime(tickers["end_data"]).dt.date

    # For some reason the cik is always interpreted as a string.
    tickers["cik"] = tickers["cik"].apply(
        lambda str: float(str) if len(str) != 0 else np.nan
    )
    return tickers

In [19]:
tickers_v1 = get_tickers(1)
tickers_v1[tickers_v1["ticker"] == "FB"]

,ticker,name,active,start_date,end_date,type,cik,composite_figi
5037,FB,"Facebook, Inc. Class A",False,2019-01-01,2021-10-29,CS,1326801.0,BBG000MM2P62
5038,FB,"Meta Platforms, Inc. Class A Common Stock",False,2021-11-01,2022-06-08,CS,1326801.0,BBG000MM2P62


In [20]:
tickers_v1[tickers_v1["ticker"] == "META"]

,ticker,name,active,start_date,end_date,type,cik,composite_figi
8462,META,"Meta Platforms, Inc. Class A Common Stock",True,2022-06-09,2023-09-01,CS,1326801.0,BBG000MM2P62


In [50]:
print(len(tickers_v1))

14779


# 3.4 Checks

1. Are SPACs handled correctly? We should expect that when they IPO a company, that they get delisted. Then one day after the delisting the new-born company should be listed. We will take a look at VFS. On 2023-8-15 it was IPO'd by the SPAC named BSAQ. So we should expect the delisting date of BSAQ to be 2023-8-14.


In [21]:
tickers_v1[tickers_v1["ticker"] == "VFS"]

,ticker,name,active,start_date,end_date,type,cik,composite_figi
13875,VFS,VinFast Auto Ltd. Ordinary Shares,True,2023-08-15,2023-09-01,CS,1913510.0,


In [22]:
tickers_v1[tickers_v1["ticker"] == "BSAQ"]

,ticker,name,active,start_date,end_date,type,cik,composite_figi
2075,BSAQ,Black Spade Acquisition Co,False,2021-09-07,2023-08-14,CS,1851908.0,


2. Let's check SVB which went bankrupt and HTZ which went from OTC to listed.

In [23]:
tickers_v1[tickers_v1["ticker"] == "SIVB"]

,ticker,name,active,start_date,end_date,type,cik,composite_figi
12081,SIVB,SVB Financial Group,False,2019-01-01,2023-03-27,CS,719739.0,BBG000BT0CM2


In [24]:
tickers_v1[tickers_v1["ticker"] == "HTZ"]

,ticker,name,active,start_date,end_date,type,cik,composite_figi
6688,HTZ,"Hertz Global Holdings, Inc.",False,2019-01-01,2020-10-29,CS,1657853.0,BBG00D5SHJH6
6689,HTZ,"Hertz Global Holdings, Inc Common Stock",True,2021-11-09,2023-09-01,CS,1657853.0,BBG011N57109


3. Sometimes tickers are re-used (e.g. META, but since it was an ETF it will not show up in our ticker list). Let's see if that has happened in our ticker list.

In [25]:
duplicated = tickers_v1[tickers_v1["ticker"].duplicated(keep=False)]
print(len(duplicated["ticker"].unique()))
print(len(duplicated))
duplicated.head()

3339
9039


,ticker,name,active,start_date,end_date,type,cik,composite_figi
0,A,Agilent Technologies,False,2019-01-01,2019-08-16,CS,1090872.0,BBG000C2V3D6
1,A,Agilent Technologies Inc.,False,2019-08-19,2022-02-07,CS,1090872.0,BBG000C2V3D6
2,A,Agilent Technologies Inc.,False,2022-02-08,2022-02-08,CS,1090872.0,BBG000BWQYZ5
3,A,Agilent Technologies Inc.,True,2022-02-09,2023-09-01,CS,1090872.0,BBG000C2V3D6
6,AAC,"AAC Holdings, Inc.",False,2019-01-01,2019-10-25,CS,1606180.0,BBG00K1Y3PT9


We will have some merging to do. However these are the "clean" ones. The next ones are just ridiculous and should not exist in the first place. 

In [26]:
from collections import Counter
print(Counter(duplicated["ticker"].values.tolist()).most_common(5))
duplicated[duplicated["ticker"] == "DGICA"].head(5)
# ???

[('CMS', 162), ('PRE', 148), ('EP', 111), ('CRESY', 24), ('DGICA', 18)]


,ticker,name,active,start_date,end_date,type,cik,composite_figi
3875,DGICA,Donegal Group Inc,False,2019-01-01,2022-08-25,CS,800457.0,BBG000JQJC22
3876,DGICA,Donegal Group Inc,False,2022-08-26,2022-08-26,CS,948046.0,BBG000JQJC22
3877,DGICA,Donegal Group Inc,False,2022-08-29,2022-09-02,CS,800457.0,BBG000JQJC22
3878,DGICA,Donegal Group Inc,False,2022-09-06,2022-09-06,CS,948046.0,BBG000JQJC22
3879,DGICA,Donegal Group Inc,False,2022-09-07,2022-09-07,CS,800457.0,BBG000JQJC22


In [27]:
duplicated[duplicated["ticker"] == "DISCK"].head(5)

,ticker,name,active,start_date,end_date,type,cik,composite_figi
3951,DISCK,"Discovery, Inc. Series C Common Stock",False,2019-01-01,2022-02-07,CS,1024333.0,
3952,DISCK,"Discovery, Inc. Series C Common Stock",False,2022-02-08,2022-02-08,CS,1024333.0,BBG000VMWHH5
3953,DISCK,"Discovery, Inc. Series C Common Stock",False,2022-02-09,2022-03-02,CS,1024333.0,
3954,DISCK,"Discovery, Inc. Series C Common Stock",False,2022-03-03,2022-03-03,CS,1024333.0,BBG000VMWHH5
3955,DISCK,"Discovery, Inc. Series C Common Stock",False,2022-03-04,2022-04-01,CS,1024333.0,


On average there are 3.3 duplicates for duplicated tickers. When we take a look it seems that it happens a lot that the name/cik/composite_figi gets changed, even though it is the same company and ticker. For example for ZWS the name is "Zurn Water Solutions Corporation" on 2022-07-01 but on the next trading day (4th July was a stock holiday) the name changes to "Zurn Elkay Water Solutions Corporation". 

# 3.5 Merging duplicates
The most straightforward way to merge these duplicates is to see for every duplicate whether the the end_date (1st occurence) and start_date (2nd occurence) are consecutive *trading days*.

In [19]:
tickers_v1.head(3)

,ticker,name,active,start_date,end_date,type,cik,composite_figi
0,A,Agilent Technologies,False,2019-01-01,2019-08-16,CS,1090872.0,BBG000C2V3D6
1,A,Agilent Technologies Inc.,False,2019-08-19,2022-02-07,CS,1090872.0,BBG000C2V3D6
2,A,Agilent Technologies Inc.,False,2022-02-08,2022-02-08,CS,1090872.0,BBG000BWQYZ5


First we need to get the duplicates.

In [21]:
###
tickers_v1 = get_tickers(1)
market_days = get_market_dates()

duplicated = tickers_v1[tickers_v1["ticker"].duplicated(keep=False)]

# Step 1: Get the indices of the rows that should be merged.
indices_duplicated = [] # looks like [['A', {1, 2, 3}], ['A', {4, 5}], ['B', {10, 11, 12, 13}]]
prev_index_and_row = None
prev_is_duplicate_and_back_to_back = False

for index, row in duplicated.iterrows():
    # Get attributes of previous ticker
    if prev_index_and_row is not None:
        prev_index = prev_index_and_row[0]
        prev_row = prev_index_and_row[1]
        prev_ticker = prev_row["ticker"]
        prev_name = prev_row["name"]
        prev_start_date = prev_row["start_date"]
        prev_end_date = prev_row["end_date"]
        prev_cik = prev_row["cik"]
        prev_figi = prev_row["composite_figi"]

    # Get attributes of current ticker
    current_index = index
    current_row = row
    current_ticker = current_row["ticker"]
    current_name = current_row["name"]
    current_start_date = current_row["start_date"]
    current_end_date = current_row["end_date"]
    current_cik = current_row["cik"]
    current_figi = current_row["composite_figi"]
    
    # Skip first index
    if prev_index_and_row is None:
        pass
    # Check if ticker duplicated and back-to-back
    elif prev_ticker == current_ticker and market_days[market_days.index(prev_end_date) + 1] == current_start_date:
        # New stock. Add stock and indices.
        if prev_is_duplicate_and_back_to_back == False:
            indices_duplicated.append([current_ticker, {prev_index, current_index}])
        # Stock already exists in indices_duplicated. Simply add indices.
        else:
            indices_duplicated[-1][-1].add(prev_index)
            indices_duplicated[-1][-1].add(current_index)
        
        # Update flag
        prev_is_duplicate_and_back_to_back = True
    else:
        prev_is_duplicate_and_back_to_back = False

    # Update prev_index_and_row for next iteration
    prev_index_and_row = (current_index, row)

In [22]:
print(len(indices_duplicated))
print(indices_duplicated[:3])

2918
[['A', {0, 1, 2, 3}], ['AADI', {12, 13}], ['AAN', {18, 19, 20, 21, 22}]]


Very rarely, it happens that the same ticker, but not the same company, has duplicates. E.g. if in our ticker list the first 5 rows is the ticker AAA, but the start and end dates are (1, 2), (2, 3), (3, 4), (9, 10), (10, 11), this means that these are two different companies. Then indices_duplicated contains ['AAA', {1, 2, 3, 4}] and ['AAA, {10, 11}]. If it *is* the same company, it managed to get delisted to OTC and revive to get their listing back. Or something is wrong with Polygons data.

In [23]:
# See which tickers are duplicated in indices_duplicated.
tickers_duplicates = set() 
for ticker, indices in indices_duplicated:
    if ticker in tickers_duplicates:
        print(ticker)
    tickers_duplicates.add(ticker)

CBL
CVU
FI
HIVE
NE
NXU
OBE
SDRL


Now that we have a list of indices of the duplicated tickers, we can merge them together. We do this by looping over <code>indices_duplicated</code> and then changing all duplicated rows to get the correct values. Then we remove the duplicates.

In [24]:
###

# Step 2: Merge duplicated in tickers_all
"""
Which value is assigned:
    name: last
    active: last
    start_date: first
    end_date: last
    type: last (but does not matter as it is always CS or ADRC)
    cik: last value that is not NaN
    compositite_figi: last value that is not NaN
"""
for ticker, indices in indices_duplicated:
    # CAUTION: Make sure that indices is sorted! Else it can happen that end_date is before start_date. I only found this out later. Moral: Always do sanity checks.
    indices = sorted(list(indices))
    ticker_data_in_tickers_v1 = tickers_v1.iloc[indices, :]
    tickers_v1.iloc[indices, tickers_v1.columns.get_loc("name")] = ticker_data_in_tickers_v1["name"].values[-1]
    tickers_v1.iloc[indices, tickers_v1.columns.get_loc("active")] = ticker_data_in_tickers_v1["active"].values[-1]
    tickers_v1.iloc[indices, tickers_v1.columns.get_loc("start_date")] = ticker_data_in_tickers_v1["start_date"].values[0]
    tickers_v1.iloc[indices, tickers_v1.columns.get_loc("end_date")] = ticker_data_in_tickers_v1["end_date"].values[-1]
    tickers_v1.iloc[indices, tickers_v1.columns.get_loc("type")] = ticker_data_in_tickers_v1["type"].values[-1]
    tickers_v1.iloc[indices, tickers_v1.columns.get_loc("cik")] = ticker_data_in_tickers_v1["cik"].ffill().values[-1]
    tickers_v1.iloc[indices, tickers_v1.columns.get_loc("composite_figi")] = ticker_data_in_tickers_v1["composite_figi"].ffill().values[-1]

tickers_v1 = tickers_v1.drop_duplicates().reset_index(drop=True)

In [25]:
len(tickers_v1)

9697

Now only a fraction of the original duplicated tickers remain.

In [26]:
duplicated = tickers_v1[tickers_v1["ticker"].duplicated(keep=False)]
print(len(duplicated["ticker"].unique()))
pd.set_option('display.max_rows', None)

duplicated.head(5)

593


,ticker,name,active,start_date,end_date,type,cik,composite_figi
3,AAC,"AAC Holdings, Inc.",False,2019-01-01,2019-10-25,CS,1606180.0,BBG00K1Y3PT9
4,AAC,Ares Acquisition Corporation,True,2021-03-25,2023-09-01,CS,1829432.0,
57,ACAC,Acies Acquisition Corp. Class A Ordinary Share,False,2020-12-11,2021-06-21,CS,1823878.0,
58,ACAC,Acri Capital Acquisition Corporation Class A C...,True,2022-08-01,2023-09-01,CS,1914023.0,BBG0160DYSM3
74,ACET,Aceto Corp,False,2019-01-01,2019-04-02,CS,2034.0,BBG00D8FFF27


We can see that some are the same company but not back-to-back. Some are different companies or went OTC and back, these are correct. However, for a lot of stocks, the first occurence only trades for a few days. That makes no sense. If you try to download data for these dates, you will see that there exists none.

So some stocks have a 'ghost' day just before their IPO. E.g. YGF was IPO'd on 2023-03-28. But on 2023-03-24 had a entry with start_date and end_date of just one day. This is the same with VCIG, which had 2 'ghost' days on 2023-03-22 and 2023-04-06. For SXTP, the ghost days were actually two. Investigating the stocks that only have 1 day in our ticker list also shows funds (that are NOT common stocks!).

Nevertheless, if start_date is equal to end_date, it's always unusable and something is wrong. So we will first remove all tickers that only exist for one day.

In [27]:
ghost_days = tickers_v1[(tickers_v1["end_date"] - tickers_v1["start_date"]) == timedelta(days=0)]
print(len(ghost_days))
ghost_days.head(5)

534


,ticker,name,active,start_date,end_date,type,cik,composite_figi
93,ACLL,"ACell, Inc. Common Stock",False,2020-07-17,2020-07-17,CS,NaN,
105,ACP,abrdn Income Credit Strategies Fund,False,2022-08-22,2022-08-22,CS,1503290.0,BBG0017VSC04
115,ACT,"Enact Holdings, Inc. Common Stock",False,2021-05-13,2021-05-13,CS,1823529.0,BBG00WSNP4R3
123,ACV,Virtus Diversified Income & Convertible Fund,False,2022-08-22,2022-08-22,CS,1636289.0,BBG008HMBD22
132,ADCT,ADC Therapeutics SA,False,2019-10-02,2019-10-02,CS,1771910.0,


In [28]:
###

# Only keep tickers that have >1 day history. But if they were just listed, keep them anyways.
tickers_v1 = tickers_v1[((tickers_v1["end_date"] - tickers_v1["start_date"]) > timedelta(days=0)) | 
                        (tickers_v1["end_date"] == market_days[market_days.index(END_DATE) - 1])]
len(tickers_v1)

9163

The remaining duplicates are:

In [29]:
duplicated = tickers_v1[tickers_v1["ticker"].duplicated(keep=False)]
print(len(duplicated["ticker"].unique()))

duplicated.head(5)

175


,ticker,name,active,start_date,end_date,type,cik,composite_figi
3,AAC,"AAC Holdings, Inc.",False,2019-01-01,2019-10-25,CS,1606180.0,BBG00K1Y3PT9
4,AAC,Ares Acquisition Corporation,True,2021-03-25,2023-09-01,CS,1829432.0,
57,ACAC,Acies Acquisition Corp. Class A Ordinary Share,False,2020-12-11,2021-06-21,CS,1823878.0,
58,ACAC,Acri Capital Acquisition Corporation Class A C...,True,2022-08-01,2023-09-01,CS,1914023.0,BBG0160DYSM3
74,ACET,Aceto Corp,False,2019-01-01,2019-04-02,CS,2034.0,BBG00D8FFF27


We need to keep in mind that the <code>start_date</code> and <code>end_date</code> may not be the start/end dates of the data. To determine the data dates, we need to loop through the ticker list and see whether the data exists. 

However, after we have downloaded our data, we can just infer it. So we will postpone this to avoid doing it twice.

# 3.6 Removing incorrect classes
There are still some weird or incorrect stock classes that we have to remove. These were found by just looking through the ticker list.

These are:
- Funds
- Preferred stock/bonds
- A "w" appended to the stock ticker
- "Ex-distribution" or "When-issued" conditions

In [30]:
###
funds = tickers_v1[tickers_v1['name'].apply(lambda s: "Fund" in s.split())]
print(len(funds))
funds.head(3)

144


,ticker,name,active,start_date,end_date,type,cik,composite_figi
122,ACV,AllianzGI Div Inc & Convert Fund,False,2019-04-02,2019-08-14,CS,1636289.0,
171,ADX,Adams Diversified Equity Fund,False,2019-04-02,2019-08-09,CS,2230.0,BBG000BB8MR6
227,AFT,Apollo Senior Floating Rate Fund Inc.,False,2019-04-02,2019-09-05,CS,1502573.0,BBG00174L007


In [31]:
###
notes = tickers_v1[tickers_v1['name'].str.contains('%')]
print(len(notes))
notes.head(3)

52


,ticker,name,active,start_date,end_date,type,cik,composite_figi
78,ACGLN,"Arch Capital Group Ltd. Depositary Shares, eac...",True,2023-05-08,2023-09-01,CS,947484.0,
256,AGNCL,AGNC Investment Corp. Depositary Shares Each R...,True,2023-05-08,2023-09-01,CS,1423689.0,
257,AGNCO,"AGNC Investment Corp. Depositary Shares, each ...",True,2023-05-08,2023-09-01,CS,1423689.0,


In [32]:
###
tickers_w = tickers_v1[tickers_v1["ticker"].str.contains("w")]
print(len(tickers_w))
tickers_w.head(3)

114


,ticker,name,active,start_date,end_date,type,cik,composite_figi
15,AANw,"The Aaron''s Company, Inc.",False,2020-11-25,2020-11-30,CS,1821393.0,BBG00WCNDCZ6
162,ADSw,Alliance Data Systems Corporation,False,2021-11-01,2021-11-05,CS,1101215.0,
302,AIRCw,Apartment Income REIT Corp.,False,2020-12-03,2020-12-14,CS,1820877.0,BBG00XK3WVD0


In [33]:
###
when_issued_or_ex_distr = tickers_v1[tickers_v1['name'].apply(lambda s: ("When" in s.split()) or ("Issued" in s.split()) or ("When-Issued" in s.split()) or ("Ex-Distribution" in s.split())  )  ]
print(len(when_issued_or_ex_distr))
when_issued_or_ex_distr.head(3)

71


,ticker,name,active,start_date,end_date,type,cik,composite_figi
136,ADEAV,Adeia Inc. Common Stock Ex-distribution When I...,False,2022-09-20,2022-09-30,CS,NaN,BBG019KN8702
534,AOUTV,"American Outdoor Brands, Inc. Common Stock Whe...",False,2020-08-10,2020-08-24,CS,1808997.0,BBG00QV8FS02
955,BBIGV,"Vinco Ventures, Inc. Ex-Distribution When-Issued",False,2022-05-17,2022-06-29,CS,1717556.0,BBG0179FCVS8


In [34]:
###
wd_suffix = tickers_v1[tickers_v1["ticker"].str.contains("\.WD")]
print(len(wd_suffix))
wd_suffix.head(3)

8


,ticker,name,active,start_date,end_date,type,cik,composite_figi
2406,DD.WD,"DuPont de Nemours, Inc",False,2021-02-02,2021-02-03,CS,1666700.0,BBG00BN961G4
2473,DHR.WD,Danaher Corporation When Distributed,False,2019-12-16,2019-12-18,CS,313616.0,BBG000BH3JF8
2720,ECL.WD,Ecolab Inc.,False,2020-06-03,2020-06-05,CS,31462.0,BBG000BHKYH4


In [35]:
###
indices_to_remove = funds.index.union(notes.index).union(tickers_w.index).union(when_issued_or_ex_distr.index).union(wd_suffix.index)

In [36]:
###
print(len(tickers_v1))
tickers_v1 = tickers_v1.drop(index=indices_to_remove)
print(len(tickers_v1))

9163
8775


Apparently this is not enough. There are still a lot of misclassified funds/preferred stocks. For example the ticker ARDC (ARES DYNAMIC CREDIT ALLOCATION FUND, INC.) is considered a common stock on 2019-04-02. However on 2019-04-01 it is considered a fund. It seems that Polygon uses the filings to determine the classification, and that common stocks are the default class. Then sometimes the filings are incomplete and Polygon classifies it as a common stock.

In [37]:
funds = tickers_v1[tickers_v1["name"].str.lower().str.contains("fund") | tickers_v1["name"].str.lower().str.contains("strategies")]
print(len(funds))
funds.head(5)

46


,ticker,name,active,start_date,end_date,type,cik,composite_figi
104,ACP,Aberdeen Income Crd Strategies,False,2019-04-02,2019-08-09,CS,1503290.0,BBG0017VSC04
618,ARDC,"ARES DYNAMIC CREDIT ALLOCATION FUND, INC.",False,2019-04-02,2019-09-05,CS,1515324.0,BBG001LJH325
685,ASFI,Asta Funding Inc,False,2019-01-01,2020-09-29,CS,1001258.0,BBG00K9VT498
1079,BGB,BLACKSTONE / GSO STRATEGIC CREDIT FUND,False,2019-04-02,2019-09-09,CS,1546429.0,BBG002W5R785
2389,DCF,BNY Mellon Alcentra Global Credit Income 2024 ...,False,2019-04-02,2019-09-09,CS,1627854.0,BBG00J2DVN20


In [38]:
client.get_ticker_details(ticker="ACP", date = "2019-04-02").type

'CS'

In [39]:
client.get_ticker_details(ticker="ACP", date = "2019-04-01").type

'FUND'

We will solve this issue by looking at the classification for 1 trading day before or after the start/end date. If it is not a common stock, we will delete it from the ticker list.

In [41]:
tickers_v1["ID"] = tickers_v1["ticker"] + '-' + tickers_v1["start_date"].astype(str)

In [ ]:
### (OPTIONAL, TAKE A LONG TIME)
IDs_to_remove = []
for index, row in tickers_v1.copy().iterrows():
    try:
        # The get_ticker_details is extremely slow.
        type_before_start = client.get_ticker_details(ticker=row['ticker'], date = row['start_date']).type
        type_after_end = client.get_ticker_details(ticker=row['ticker'], date = row['end_date']).type
    except Exception as e:
        repr(e)
        continue
    
    print(index)

    if type_before_start != 'CS' or type_after_end != 'CS':
        IDs_to_remove.append(row['ID'])

# Write to file. We need this later to make updating easier.
import csv
with open(POLYGON_DATA_PATH + f'raw/IDs_to_remove.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerow(IDs_to_remove)

In [43]:
###
with open(POLYGON_DATA_PATH + f'raw/IDs_to_remove.csv', 'r') as file:
    reader = csv.reader(file)
    IDs_to_remove = next(reader)

print(len(tickers_v1))
tickers_v1 = tickers_v1[~tickers_v1['ID'].isin(IDs_to_remove)]
tickers_v1.reset_index(inplace=True, drop=True)
print(len(tickers_v1))

8775
8167


This is a ridiculous amount of misclassified common stocks.

Finally, we save the merged and cleaned ticker list to <code>tickers_v2.csv</code>. Also we add an "ID" column.

In [44]:
###
print(f"Total tickers: {len(tickers_v1)}")
print(f"Unique tickers: {len(tickers_v1['ticker'].unique())}")

tickers_v1 = tickers_v1.reset_index(drop=True)
tickers_v1 = tickers_v1[["ID", "ticker", "name", "active", "start_date", "end_date", "type", "cik", "composite_figi"]]
tickers_v1.to_csv("../data/tickers_v2.csv")

Total tickers: 8167
Unique tickers: 8012


We now have only half of the original tickers remaining. The other half were duplicates and incorrect classified stocks.

---

That was a **lot** of work for *just* getting a list of tickers. This has almost become more of a data engineering project than research. But now we can be sure that our data does not contain survivorship bias. And (hopefully) had the correct start and end dates for downloading data. These may still be incorrect tickers (such as a random fund because Polygon ticker lists suck). 

If the <code>Ticker Events</code> included delistings companies and all name changes, this would be much easier. Or if the ticker list was complete had the start dates included. Or if all classifications were correct. In the future I hope this notebook becomes obsolete. But it is unlikely.

---

# 3.7 Updates
1. Update END_DATE.
2. Run the 3 ### cells in 3.1. This updates the folder of ticker lists. It takes around 7 seconds per day.
3. Run the cell below to update <code>tickers_v1</code>. Instead of merging all ticker lists, only the new ones are merged.
4. Run the cells with ### in 3.5 and 3.6 to update <code>tickers_v2</code>. We skip checking if a stock is a common stock is not because it is very time consuming. If you do not want to skip it, run the '### (OPTIONAL)' row.

In [12]:
from utils import get_tickers

tickers_v1 = get_tickers(1, cik_as_float=False)
current_end_date = tickers_v1['end_date'].max()

market_days = get_market_dates()
first_trading_date_after_current_end_date = first_trading_date_after_equal(current_end_date + timedelta(days=1))
last_trading_date_before_end_date = last_trading_date_before_equal(END_DATE)

for day in market_days:
    if day == current_end_date:
        our_tickers = tickers_v1

    elif day >= first_trading_date_after_current_end_date and day <= END_DATE:
        # Get new ticker list to update ours
        tickers_day_i = pd.read_csv(
            POLYGON_DATA_PATH + f"raw/tickers/{day.isoformat()}.csv",
            index_col=0,
            keep_default_na=False,
            na_values=['#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND', '1.#QNAN', '<NA>', 'N/A', 'NULL', 'NaN', 'None', 'n/a', 'nan', 'null']
        )
        tickers_day_i = tickers_day_i[["ticker", "name", "active", "cik", "composite_figi", "type"]]
        tickers_day_i = tickers_day_i[tickers_day_i["active"] == True]
        tickers_day_i.reset_index(inplace=True, drop=True)

        # Preliminary check: no duplicates
        if our_tickers[["ticker", "name", "active", "cik", "composite_figi", "type"]].duplicated().all():
            raise Exception("There are duplicates!")

        if tickers_day_i[["ticker", "name", "active", "cik", "composite_figi", "type"]].duplicated().all():
            raise Exception("There are duplicates!")

        # DELISTINGS
        indicator_delisted = our_tickers[["ticker", "name", "active", "cik", "composite_figi", "type"]].merge(tickers_day_i[["ticker", "name", "active", "cik", "composite_figi", "type"]], on=["ticker", "name", "active", "cik", "composite_figi", "type"], how='left', indicator=True)

        indicator_delisted['_merge'] = np.where(our_tickers["active"], indicator_delisted['_merge'], "both") # ERROR FIX: If in our ticker list we have already set it inactive, it should not be added to the list of delisted stocks again. By setting _merge to "both" we skip the already inactive stocks.

        indicator_delisted = indicator_delisted["_merge"] # Only get the indicator
        delisted_tickers = our_tickers[indicator_delisted == "left_only"]

        # NEW LISTINGS
        indicator_new = tickers_day_i[["ticker", "name", "active", "cik", "composite_figi", "type"]].merge(our_tickers[["ticker", "name", "active", "cik", "composite_figi", "type"]], on=["ticker", "name", "active", "cik", "composite_figi", "type"], 
                        how='left', indicator=True)
        indicator_new = indicator_new["_merge"]
        new_tickers = tickers_day_i[indicator_new == "left_only"]

        # PROCESS DELISTINGS
        previous_day = market_days[market_days.index(day) - 1] # Getting previous trading day
        our_tickers.loc[indicator_delisted == "left_only", "end_date"] = previous_day
        our_tickers.loc[indicator_delisted == "left_only", "active"] = False
        
        # PROCESS NEW LISTINGS
        our_tickers = pd.concat([our_tickers, new_tickers])

        our_tickers.reset_index(inplace=True, drop=True)
        our_tickers['start_date'].fillna(value=day, inplace=True)
        
        # Final checks
        if our_tickers[["ticker", "name", "active", "type", "start_date"]].isnull().values.any():
            #null_data = our_tickers[our_tickers[["ticker", "name", "active", "type", "start_date"]].isnull().any(axis=1)]
            raise Exception("There are missing values.")
        
        print(f'{day.isoformat()}: Amount of stocks {len(our_tickers)}')
        
        # Finalize
        if day == last_trading_date_before_end_date:
            our_tickers["end_date"].fillna(value=last_trading_date_before_end_date, inplace=True)
            our_tickers = our_tickers.sort_values(by=["ticker", "end_date"]).reset_index(drop=True)
            our_tickers[["ticker", "name", "active", "start_date", "end_date", "type", "cik", "composite_figi"]].to_csv("../data/tickers_v1.csv")

2023-09-05: Amount of stocks 14794
2023-09-06: Amount of stocks 14799
2023-09-07: Amount of stocks 14802
